In [30]:
# Imports de modules du projet
import config, src

# Imports classiques
import numpy as np
import pandas as pd

# Importes spécifiques
import holidays

In [31]:
data = src.load_data(config.RAW_DATA_FILE)
data['date'] = pd.to_datetime(data['date'])

Données chargées avec succès.


In [32]:
rename_dict = {
    "date": "Date",
    "Appliances": "Energy",
    "lights": "Lights",
    "T1": "Kitchen_T",
    "RH_1": "Kitchen_H",
    "T2": "Living_T",
    "RH_2": "Living_H",
    "T3": "Laundry_T",
    "RH_3": "Laundry_H",
    "T4": "Office_T",
    "RH_4": "Office_H",
    "T5": "Bath_T",
    "RH_5": "Bath_H",
    "T6": "Outside_T",
    "RH_6": "Outside_H",
    "T7": "Ironing_T",
    "RH_7": "Ironing_H",
    "T8": "TeenRoom_T",
    "RH_8": "TeenRoom_H",
    "T9": "ParentsRoom_T",
    "RH_9": "ParentsRoom_H",
    "T_out": "Weather_T",
    "Pressure": "Weather_P",
    "RH_out": "Weather_H",
    "Windspeed": "Weather_WindSpeed",
    "Visibility": "Weather_Visibility",
    "Tdewpoint": "Weather_Dewpoint",
    "rv1": "RandVar_1",
    "rv2": "RandVar_2"
}

data.rename(columns=rename_dict, inplace=True)

In [33]:
src.save_data(data, 'renamed_data')

Données sauvegardées avec succès.


In [34]:
days = {
    0: 'monday',
    1: 'tuesday',
    2: 'wednesday',
    3: 'thursday',
    4: 'friday',
    5: 'saturday',
    6: 'sunday'
}

days.values()

dict_values(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])

In [35]:
data['day_of_week'] = data['date'].dt.weekday
data['day_of_week'] = data['day_of_week'].map(days)

one_hot = pd.get_dummies(data['day_of_week'], columns=['day_of_week'], prefix='', prefix_sep='')
one_hot = one_hot[list(days.values())]

data = pd.concat([data.drop(columns=['day_of_week']), one_hot], axis=1)
data[list(days.values())] = data[list(days.values())].astype(int)
data.info()

KeyError: 'date'

In [33]:
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,Tdewpoint,rv1,rv2,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,5.3,13.275433,13.275433,1,0,0,0,0,0,0
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,5.2,18.606195,18.606195,1,0,0,0,0,0,0
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,5.1,28.642668,28.642668,1,0,0,0,0,0,0
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,5.0,45.410389,45.410389,1,0,0,0,0,0,0
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,4.9,10.084097,10.084097,1,0,0,0,0,0,0


In [34]:
data['hour'] = data['date'].dt.hour
data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 24)
data['cos_hour'] = np.cos(2 * np.pi * data['hour'] / 24)

data = data.drop(columns=['hour'])

In [35]:
be_holidays = holidays.Belgium(years=data['date'].dt.year.unique())
be_holidays = {pd.Timestamp(date): name for date, name in be_holidays.items()}
data['is_holiday'] = data['date'].isin(be_holidays).astype(int)

n_holidays = data[data['is_holiday'] == 1]['date'].count()
print(f'Il a {n_holidays} jour(s) férié(s) dans le dataset.')

Il a 6 jour(s) férié(s) dans le dataset.


In [36]:
be_school_holidays_2016 = [
    ("2016-02-08", "2016-02-14"),  # Carnaval
    ("2016-03-28", "2016-04-10"),  # Pâques
    ("2016-07-01", "2016-08-31"),  # Été
    ("2016-10-31", "2016-11-06"),  # Toussaint
    ("2016-12-26", "2017-01-08")   # Noël
]

def is_school_holiday(date, holidays):
    for start, end in holidays:
        if pd.Timestamp(start) <= date <= pd.Timestamp(end):
            return True
    return False

In [37]:
data['is_school_holiday'] = data['date'].apply(lambda x: is_school_holiday(x, be_school_holidays_2016)).astype(int)

n_school_holidays = len(data.loc[data['is_school_holiday'] == True, 'date'].dt.date.unique())
print(f'Il y a {n_school_holidays} jour(s) de vacances scolaires dans le dataset.')

Il y a 21 jour(s) de vacances scolaires dans le dataset.


In [38]:
def get_season(date):
    year = date.year
    seasons = {
        0: (pd.Timestamp(year=year, month=3, day=21), pd.Timestamp(year=year, month=6, day=20)), # Printemps
        1: (pd.Timestamp(year=year, month=6, day=21), pd.Timestamp(year=year, month=9, day=20)), # Été
        2: (pd.Timestamp(year=year, month=9, day=21), pd.Timestamp(year=year, month=12, day=20)), # Automne
        3: (pd.Timestamp(year=year, month=12, day=21), pd.Timestamp(year=year+1, month=3, day=20)) # Hiver
        }
    for season, (start, end) in seasons.items():
        if start <= date <= end:
            return season
    return 3 # 01/01 - 20/03

In [39]:
index_to_season = {
    0: 'spring',
    1: 'summer',
    2: 'fall',
    3: 'winter'
}

In [40]:
data['season'] = data['date'].apply(get_season)

n_seasons = len(data['season'].unique())
print(f'Il y a {n_seasons} différentes dans le dataset.')

Il y a 2 différentes dans le dataset.


In [41]:
data['season'].value_counts()

season
3    9978
0    9757
Name: count, dtype: int64

In [42]:
encode_winter_spring = {
    0: 0,
    3: 1
}

data['season'] = data['season'].replace(encode_winter_spring)

In [43]:
data['season'].value_counts()

season
1    9978
0    9757
Name: count, dtype: int64

In [44]:
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,wednesday,thursday,friday,saturday,sunday,sin_hour,cos_hour,is_holiday,is_school_holiday,season
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,0,0,0,0,0,-0.965926,-0.258819,0,0,1


In [45]:
data.duplicated().value_counts()

False    19735
Name: count, dtype: int64

In [46]:
save_data(data, 'processed_data')

Données sauvegardées avec succès.
